In [1]:
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [4]:
from data.metrics import recall
from visualization.plot_utils import write_latex_table, summary_pooling_table, \
pooling_plots, pandas_settings, plot_settings,  save_result, glance_dict
# general
import pandas as pd
import numpy as np
from tqdm import tqdm
from features.tracks_info import TrackInfo
from collections import OrderedDict
import compressed_pickle as cpick
from os.path import join as path_join
from gensim import corpora
import logging, gensim
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

SAVE_PATH='../data/genre_lda'

# create pooling model

In [3]:
turi2auri = cpick.load('../data/interim/track_uri2artist_uri.pkl.bz2')
genres = cpick.load('../data/interim/genres.pkl.bz2')
turi2genres ={}
for turi,auri in turi2auri.items():
    turi2genres[turi]=genres[auri]
#cpick.save('../data/interim/track_uri2genres.pkl.bz2',turi2genres)
glance_dict(turi2genres)

{'0UaMYEvWZi0ZqiDOoHU3YI': ['dance pop',
  'hip hop',
  'hip pop',
  'pop',
  'pop rap',
  'r&b',
  'southern hip hop',
  'urban contemporary'],
 '0WqIKmW4BTrj3eJFmnCKMv': ['dance pop', 'pop', 'post-teen pop', 'r&b'],
 '6I9VzXrHxO9rA9A5euc8Ak': ['dance pop',
  'pop',
  'post-teen pop',
  'urban contemporary']}

## prune and vectorize words

In [6]:
texts = [val for val in turi2genres.values()]

## dictionary and mm data

In [7]:
dictionary = corpora.Dictionary(texts)
dictionary.save(path_join(SAVE_PATH,'genres.dict'))
print(glance_dict(dictionary.token2id,10))
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(path_join(SAVE_PATH,'genres.mm'), corpus)

{'dance pop': 0, 'hip hop': 1, 'hip pop': 2, 'pop': 3, 'pop rap': 4, 'r&b': 5, 'southern hip hop': 6, 'urban contemporary': 7, 'post-teen pop': 8, 'reggae fusion': 9}


## Train model

In [8]:

#id2word = gensim.corpora.Dictionary.load_from_text('genres.dict')
mm = gensim.corpora.MmCorpus(path_join(SAVE_PATH,'genres.mm'))
lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=dictionary,
                                      alpha='auto', eta='auto', 
                                      num_topics=20, update_every=1,
                                      chunksize=10000, passes=5)
lda.save(path_join(SAVE_PATH,'genre_lda'))

2018-05-09 00:22:35,461 : INFO : loaded corpus index from ../data/genre_lda/genres.mm.index
2018-05-09 00:22:35,462 : INFO : initializing cython corpus reader from ../data/genre_lda/genres.mm
2018-05-09 00:22:35,463 : INFO : accepted corpus with 2262292 documents, 1686 features, 6507084 non-zero entries
2018-05-09 00:22:35,463 : INFO : using autotuned alpha, starting with [0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001, 0.050000001]
2018-05-09 00:22:35,464 : INFO : using serial LDA version on this node
2018-05-09 00:22:35,470 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 2262292 documents, updating model once every 10000 documents, evaluating perplexity every 100000 documents, iterating 50x with a convergence threshold of 0.001000
2018-05

2018-05-09 00:22:39,946 : INFO : topic diff=3.505553, rho=0.500000
2018-05-09 00:22:39,977 : INFO : PROGRESS: pass 0, at document #50000/2262292
2018-05-09 00:22:40,777 : INFO : optimized alpha [0.050802764, 0.041004047, 0.046092998, 0.041727066, 0.042066906, 0.042395037, 0.044637807, 0.045703221, 0.043023273, 0.047410473, 0.042021416, 0.045169812, 0.043237809, 0.041725386, 0.044116426, 0.043104589, 0.047085296, 0.043508787, 0.043230064, 0.042854927]
2018-05-09 00:22:40,777 : INFO : merging changes from 10000 documents into a model of 2262292 documents
2018-05-09 00:22:40,781 : INFO : topic #1 (0.041): 0.046*"post-hardcore" + 0.035*"dance rock" + 0.034*"new wave" + 0.032*"post-punk" + 0.031*"alternative rock" + 0.031*"uk post-punk" + 0.024*"experimental rock" + 0.023*"power pop" + 0.023*"art rock" + 0.022*"canadian indie"
2018-05-09 00:22:40,781 : INFO : topic #3 (0.042): 0.065*"modern rock" + 0.058*"comic" + 0.051*"rock" + 0.045*"alternative rock" + 0.044*"turkish pop" + 0.043*"perman

2018-05-09 00:22:43,873 : INFO : topic #3 (0.039): 0.090*"otacore" + 0.080*"rock" + 0.073*"permanent wave" + 0.060*"alternative rock" + 0.059*"comic" + 0.057*"modern rock" + 0.038*"antiviral pop" + 0.030*"garage rock" + 0.028*"indie rock" + 0.027*"a cappella"
2018-05-09 00:22:43,874 : INFO : topic #9 (0.045): 0.152*"edm" + 0.112*"electro house" + 0.077*"big room" + 0.077*"tropical house" + 0.064*"pop" + 0.057*"brostep" + 0.054*"progressive electro house" + 0.048*"progressive house" + 0.041*"electronic trap" + 0.036*"house"
2018-05-09 00:22:43,874 : INFO : topic #16 (0.047): 0.134*"pop" + 0.126*"dance pop" + 0.111*"post-teen pop" + 0.051*"indietronica" + 0.039*"synthpop" + 0.038*"metropopolis" + 0.035*"alternative dance" + 0.033*"new rave" + 0.033*"indie poptimism" + 0.026*"indie r&b"
2018-05-09 00:22:43,875 : INFO : topic #0 (0.052): 0.146*"rap" + 0.133*"hip hop" + 0.129*"pop rap" + 0.093*"southern hip hop" + 0.074*"trap music" + 0.068*"gangster rap" + 0.051*"dirty south rap" + 0.042*"

2018-05-09 00:22:47,985 : INFO : topic #16 (0.047): 0.116*"dance pop" + 0.113*"pop" + 0.092*"post-teen pop" + 0.051*"indietronica" + 0.042*"synthpop" + 0.041*"alternative dance" + 0.040*"new rave" + 0.036*"metropopolis" + 0.035*"indie poptimism" + 0.026*"vapor soul"
2018-05-09 00:22:47,986 : INFO : topic #0 (0.051): 0.139*"rap" + 0.128*"hip hop" + 0.120*"pop rap" + 0.090*"southern hip hop" + 0.073*"gangster rap" + 0.073*"trap music" + 0.055*"dirty south rap" + 0.046*"hardcore hip hop" + 0.036*"alternative hip hop" + 0.035*"pop"
2018-05-09 00:22:47,986 : INFO : topic diff=1.271005, rho=0.277350
2018-05-09 00:22:48,007 : INFO : PROGRESS: pass 0, at document #140000/2262292
2018-05-09 00:22:48,695 : INFO : optimized alpha [0.051156316, 0.036826741, 0.042231519, 0.036994163, 0.038786374, 0.039043088, 0.043448314, 0.043240625, 0.040758338, 0.043362785, 0.039150611, 0.045820415, 0.040864963, 0.039337859, 0.04115114, 0.041782044, 0.046578847, 0.04048847, 0.042401638, 0.04076042]
2018-05-09 00

2018-05-09 00:22:51,509 : INFO : optimized alpha [0.050138291, 0.036180396, 0.041161649, 0.035917208, 0.038366638, 0.038048763, 0.043434452, 0.043133896, 0.040713973, 0.042179767, 0.038184565, 0.045611262, 0.040397864, 0.038861971, 0.039943371, 0.041684978, 0.046171308, 0.039676342, 0.041917603, 0.040316042]
/home/beangoben/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
2018-05-09 00:22:51,517 : INFO : merging changes from 10000 documents into a model of 2262292 documents
2018-05-09 00:22:51,520 : INFO : topic #3 (0.036): 0.120*"permanent wave" + 0.109*"rock" + 0.083*"comic" + 0.067*"modern rock" + 0.061*"alternative rock" + 0.041*"otacore" + 0.038*"compositional ambient" + 0.035*"garage rock" + 0.034*"a cappella" + 0.028*"antiviral pop"
2018-05-09 00:22:51,521 : INFO : topic #1 (0.036): 0.059*"punk" + 0.048*"new wave" + 0.039*"dance rock" + 0.038*"alternative rock" + 0.036*"ska pu

...

2018-05-09 00:36:34,458 : INFO : topic #13 (0.043): 0.231*"classical" + 0.118*"romantic era" + 0.059*"scorecore" + 0.057*"early music" + 0.054*"classical era" + 0.052*"soundtrack" + 0.045*"baroque" + 0.040*"video game music" + 0.032*"rock-and-roll" + 0.032*"rockabilly"
2018-05-09 00:36:34,459 : INFO : topic #15 (0.046): 0.067*"adult standards" + 0.065*"christmas" + 0.060*"vocal jazz" + 0.050*"jazz" + 0.045*"bebop" + 0.044*"lounge" + 0.044*"cool jazz" + 0.044*"jazz blues" + 0.043*"swing" + 0.039*"big band"
2018-05-09 00:36:34,459 : INFO : topic #8 (0.049): 0.036*"soul blues" + 0.034*"soul" + 0.033*"jazz blues" + 0.032*"funk" + 0.029*"jazz funk" + 0.027*"dancehall" + 0.026*"soul jazz" + 0.025*"classic funk rock" + 0.025*"roots reggae" + 0.025*"motown"
2018-05-09 00:36:34,460 : INFO : topic diff=inf, rho=0.065763
2018-05-09 00:36:34,473 : INFO : PROGRESS: pass 4, at document #2070000/2262292
2018-05-09 00:36:34,975 : INFO : optimized alpha [0.026274009, 0.032824136, 0.020591293, 0.0298403

2018-05-09 00:36:37,531 : INFO : topic diff=inf, rho=0.065763
2018-05-09 00:36:37,547 : INFO : PROGRESS: pass 4, at document #2110000/2262292
2018-05-09 00:36:38,042 : INFO : optimized alpha [0.026186448, 0.032852836, 0.02054053, 0.029907135, 0.037752781, 0.02352575, 0.042147625, 0.029929509, 0.049184792, 0.022652827, 0.038982734, 0.036301684, 0.031511303, 0.043591954, 0.022803221, 0.046040345, 0.027816996, 0.037868436, 0.028293576, 0.033195831]
2018-05-09 00:36:38,043 : INFO : merging changes from 10000 documents into a model of 2262292 documents
2018-05-09 00:36:38,047 : INFO : topic #2 (0.021): 0.117*"emo" + 0.099*"pop punk" + 0.082*"contemporary country" + 0.075*"country road" + 0.068*"country" + 0.049*"screamo" + 0.044*"redneck" + 0.042*"dreamo" + 0.042*"modern country rock" + 0.040*"brazilian rock"
2018-05-09 00:36:38,048 : INFO : topic #9 (0.023): 0.111*"progressive house" + 0.093*"trance" + 0.091*"progressive trance" + 0.076*"edm" + 0.058*"house" + 0.051*"big room" + 0.050*"ele

2018-05-09 00:36:40,134 : INFO : topic #9 (0.023): 0.110*"progressive house" + 0.089*"trance" + 0.088*"progressive trance" + 0.076*"edm" + 0.060*"house" + 0.052*"electro house" + 0.050*"big room" + 0.041*"disco house" + 0.035*"tropical house" + 0.033*"progressive electro house"
2018-05-09 00:36:40,134 : INFO : topic #13 (0.044): 0.240*"classical" + 0.118*"romantic era" + 0.064*"early music" + 0.055*"scorecore" + 0.053*"baroque" + 0.053*"soundtrack" + 0.052*"classical era" + 0.037*"video game music" + 0.032*"german jazz" + 0.030*"rockabilly"
2018-05-09 00:36:40,135 : INFO : topic #15 (0.046): 0.063*"adult standards" + 0.063*"christmas" + 0.057*"vocal jazz" + 0.049*"jazz" + 0.046*"cool jazz" + 0.046*"bebop" + 0.042*"lounge" + 0.041*"jazz blues" + 0.040*"swing" + 0.035*"big band"
2018-05-09 00:36:40,135 : INFO : topic #8 (0.049): 0.034*"soul blues" + 0.032*"jazz blues" + 0.032*"soul" + 0.029*"funk" + 0.027*"jazz funk" + 0.027*"blues" + 0.026*"dancehall" + 0.026*"roots reggae" + 0.025*"ear

2018-05-09 00:36:42,602 : INFO : topic diff=inf, rho=0.065763
2018-05-09 00:36:43,479 : INFO : -20.592 per-word bound, 1581088.3 perplexity estimate based on a held-out corpus of 10000 documents with 20942 words
2018-05-09 00:36:43,479 : INFO : PROGRESS: pass 4, at document #2200000/2262292
2018-05-09 00:36:43,964 : INFO : optimized alpha [0.02600329, 0.032993052, 0.020399095, 0.030023661, 0.037810493, 0.02345744, 0.042076319, 0.029813061, 0.049619086, 0.022588756, 0.039077621, 0.036127478, 0.031620082, 0.043931831, 0.022807408, 0.046247888, 0.027748113, 0.037894271, 0.028223593, 0.033024952]
2018-05-09 00:36:43,965 : INFO : merging changes from 10000 documents into a model of 2262292 documents
2018-05-09 00:36:43,969 : INFO : topic #2 (0.020): 0.105*"emo" + 0.097*"pop punk" + 0.079*"contemporary country" + 0.074*"country road" + 0.063*"country" + 0.049*"brazilian rock" + 0.048*"redneck" + 0.045*"country dawn" + 0.044*"dreamo" + 0.044*"screamo"
2018-05-09 00:36:43,969 : INFO : topic #9

2018-05-09 00:36:46,030 : INFO : topic #9 (0.023): 0.110*"progressive house" + 0.094*"trance" + 0.093*"progressive trance" + 0.074*"edm" + 0.060*"house" + 0.051*"electro house" + 0.048*"big room" + 0.041*"disco house" + 0.034*"crossover thrash" + 0.033*"tropical house"
2018-05-09 00:36:46,031 : INFO : topic #13 (0.044): 0.240*"classical" + 0.122*"romantic era" + 0.060*"early music" + 0.059*"classical era" + 0.054*"scorecore" + 0.054*"soundtrack" + 0.048*"baroque" + 0.037*"video game music" + 0.030*"rockabilly" + 0.027*"rock-and-roll"
2018-05-09 00:36:46,031 : INFO : topic #15 (0.046): 0.072*"adult standards" + 0.071*"christmas" + 0.061*"vocal jazz" + 0.049*"jazz" + 0.049*"lounge" + 0.043*"bebop" + 0.043*"jazz blues" + 0.042*"cool jazz" + 0.041*"swing" + 0.034*"big band"
2018-05-09 00:36:46,032 : INFO : topic #8 (0.050): 0.036*"soul blues" + 0.034*"soul" + 0.032*"jazz blues" + 0.032*"funk" + 0.027*"dancehall" + 0.027*"jazz funk" + 0.026*"disco" + 0.025*"classic funk rock" + 0.025*"roots

In [10]:
cpick.save(path_join(SAVE_PATH,'lda_topics_explained'),{-1:'Topicless',
                                                        0:'Rap/Hip Hop',
                                                        1:'punk-like',
                                                        2:'emo-like',
                                                        3:'ambient/foreign-like',
                                                       4:'latin-dance-like',
                                                       5:'worship/rock',
                                                       6:'rock-like',
                                                       7:'metal-like',
                                                       8:'blues',
                                                       9:'house',
                                                       10:'latin-rock',
                                                       11:'indie-rock',
                                                       12:'foreign',
                                                       13:'instrumental',
                                                       14:'country',
                                                       15:'jazz',
                                                       16:'electronic',
                                                       17:'new_wave',
                                                       18:'pop-rock',
                                                       19:'dance-pop'})


Saving to ../data/genre_lda/lda_topics_explained.pkl.bz2


## test model

In [7]:
dictionary = gensim.corpora.Dictionary.load(path_join(SAVE_PATH,'genres.dict'))
lda = gensim.models.ldamodel.LdaModel.load(path_join(SAVE_PATH,'genre_lda'))
doc_bow = dictionary.doc2bow(turi2genres['0UaMYEvWZi0ZqiDOoHU3YI'])
topic_mix= lda[doc_bow]
print(topic_mix)

2018-05-09 01:14:35,758 : INFO : loading Dictionary object from ../data/genre_lda/genres.dict
2018-05-09 01:14:35,759 : INFO : loaded ../data/genre_lda/genres.dict
2018-05-09 01:14:35,760 : INFO : loading LdaModel object from ../data/genre_lda/genre_lda
2018-05-09 01:14:35,761 : INFO : loading expElogbeta from ../data/genre_lda/genre_lda.expElogbeta.npy with mmap=None
2018-05-09 01:14:35,762 : INFO : setting ignored attribute dispatcher to None
2018-05-09 01:14:35,763 : INFO : setting ignored attribute id2word to None
2018-05-09 01:14:35,764 : INFO : setting ignored attribute state to None
2018-05-09 01:14:35,765 : INFO : loaded ../data/genre_lda/genre_lda
2018-05-09 01:14:35,765 : INFO : loading LdaState object from ../data/genre_lda/genre_lda.state
2018-05-09 01:14:35,766 : INFO : loaded ../data/genre_lda/genre_lda.state


NameError: name 'turi2genres' is not defined

In [21]:
turi2genres = cpick.load('../data/interim/track_uri2genres.pkl.bz2')
turi2gtopic={}
for turi,genres in turi2genres.items():
    topic_mix= lda[dictionary.doc2bow(genres)]
    w = np.zeros(20)
    for indx,val in topic_mix:
        w[indx]=val
    turi2gtopic[turi]=w
print(glance_dict(turi2gtopic))
cpick.save('../data/interim/track_uri2gtopics_vec.pkl.bz2',turi2gtopic)

{'0UaMYEvWZi0ZqiDOoHU3YI': array([ 0.38311425,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.54725027]), '6I9VzXrHxO9rA9A5euc8Ak': array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.01069641,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.86506933]), '0WqIKmW4BTrj3eJFmnCKMv': array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.01069641,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.86506933])}
Saving to ../data/interim/track_uri2gtopics_vec.pkl

In [15]:
turi2genres = cpick.load('../data/interim/track_uri2genres.pkl.bz2')
turi2gtopic={}
for turi,genres in turi2genres.items():
    topic_mix= lda[dictionary.doc2bow(genres)]
    w = [i[1] for i in topic_mix]
    w_mean = np.mean(w)
    w_max = np.max(w)
    if  w_mean+0.01 > w_max:
        topic=-1
    else:
        topic =  [i[0] for i in topic_mix if i[1]==w_max][0]
        
    turi2gtopic[turi]=topic
print(glance_dict(turi2gtopic))
cpick.save('../data/interim/track_uri2gtopics_max.pkl.bz2',turi2gtopic)

{'0UaMYEvWZi0ZqiDOoHU3YI': 19, '6I9VzXrHxO9rA9A5euc8Ak': 19, '0WqIKmW4BTrj3eJFmnCKMv': 19}
Saving to ../data/interim/track_uri2gtopics_max.pkl.bz2
